In [313]:
import requests
import json
import pandas as  pd
import time
# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

In [314]:
url = 'http://127.0.0.1:5000/createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [315]:
response.json()

{'signingKey_hex': '82c44010c6b84b8b735b945d4c47513ae4a628ba5c39dc3816dbb915e7ef6770',
 'traderId': '1',
 'verifyKey_hex': '374b7124e5c693c58fb13b6715e5214255b405f5ebf8fe4c71746c826ace50f9'}

In [316]:
url = 'http://127.0.0.1:5000/createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [317]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [318]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,1,1


In [319]:
# Create a trade
url = 'http://127.0.0.1:5000/createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [320]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [321]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,1,0.551,1,1


In [322]:
# Create a matched trade
url = 'http://127.0.0.1:5000/createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [323]:
response.json()

{'checks': '(True, True)',
 'marketBranchId': 1,
 'marketRootId': 1,
 'price': 0.551,
 'quantity': -1,
 'traderId': '1'}

In [324]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 1.2269649505615234 seconds.
matched trade: 1 in 1.229762077331543 seconds.
matched trade: 2 in 1.2471270561218262 seconds.
matched trade: 3 in 1.2961277961730957 seconds.
matched trade: 4 in 1.293416976928711 seconds.
matched trade: 5 in 1.364065170288086 seconds.
matched trade: 6 in 1.402085781097412 seconds.
matched trade: 7 in 1.4071826934814453 seconds.
matched trade: 8 in 1.5178959369659424 seconds.
matched trade: 9 in 1.3795678615570068 seconds.


In [325]:
url = 'http://127.0.0.1:5000/createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 0.22948098182678223 seconds.
market created: 2 in 0.2810397148132324 seconds.
market created: 3 in 0.4254038333892822 seconds.
market created: 4 in 0.8604011535644531 seconds.


In [326]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 1}

In [327]:
url = 'http://127.0.0.1:5000/viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [328]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,1,1,0,0.7
1,2,1,0,0.7
2,3,1,0,0.7
3,4,1,0,0.7


In [329]:
url = 'http://127.0.0.1:5000/viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [330]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.551,1,1
1,1,1,0.551,-1,1
10,1,1,0.551,1,1
11,1,1,0.551,-1,1
12,1,1,0.551,-1,1
13,1,1,0.551,1,1
14,1,1,0.551,-1,1
15,1,1,0.551,1,1
16,1,1,0.551,1,1
17,1,1,0.551,-1,1


In [336]:
# Create many matched trades
url = 'http://127.0.0.1:5000/createTrade'

for i in range(1000):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.01
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 1.32413911819458 seconds.
matched trade: 1 in 1.2459490299224854 seconds.
matched trade: 2 in 1.238539695739746 seconds.
matched trade: 3 in 1.2344379425048828 seconds.
matched trade: 4 in 1.2509098052978516 seconds.
matched trade: 5 in 1.2376270294189453 seconds.
matched trade: 6 in 1.2815358638763428 seconds.
matched trade: 7 in 1.2580442428588867 seconds.
matched trade: 8 in 1.280808925628662 seconds.
matched trade: 9 in 1.2728493213653564 seconds.
matched trade: 10 in 1.271353006362915 seconds.
matched trade: 11 in 1.2957491874694824 seconds.
matched trade: 12 in 1.2826220989227295 seconds.
matched trade: 13 in 1.2599680423736572 seconds.
matched trade: 14 in 1.2396998405456543 seconds.
matched trade: 15 in 1.2618708610534668 seconds.
matched trade: 16 in 1.295686960220337 seconds.
matched trade: 17 in 1.2821049690246582 seconds.
matched trade: 18 in 1.2624671459197998 seconds.
matched trade: 19 in 1.2936060428619385 seconds.
matched trade: 20 in 1.2725689411163

In [332]:
url = 'http://127.0.0.1:5000/viewOpenTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [333]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.01,25,1


In [334]:
url = 'http://127.0.0.1:5000/viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [335]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,1,1,0.551,0,1
